In [1]:
# Databricks notebook source
# MAGIC %md
# MAGIC # PRODUCTION ETL - v3.2
# MAGIC
# MAGIC **Changes from v3.1 (single change — incremental):**
# MAGIC - FIX: audit_trail rows column type conflict (INT vs BIGINT) resolved correctly
# MAGIC - ROOT CAUSE: ALTER TABLE CHANGE COLUMN for type widening is unsupported in this
# MAGIC   Delta Lake protocol version. mergeSchema=true handles additive changes only —
# MAGIC   it does not resolve type conflicts. Both approaches therefore failed.
# MAGIC - SOLUTION: Cast incoming rows value to IntegerType at write time to match the
# MAGIC   existing table schema. Eliminates the type conflict without DDL surgery.
# MAGIC   INT is safe for row counts at current 2.7M scale. When scale approaches 2.1B
# MAGIC   rows, schedule a planned FORCE_RECREATE during a maintenance window.
# MAGIC - REMOVED: ALTER TABLE CHANGE COLUMN attempt (unsupported, caused silent fallthrough)
# MAGIC - REMOVED: mergeSchema=true on audit_trail write (masks type conflicts, not a fix)
# MAGIC - Bronze table strategy note: MERGE on person_id is correct for a full extract pattern.
# MAGIC   Source deletions are not yet handled — tracked as pending item for v4.0.

# COMMAND ----------

import pyspark.sql.functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable
from datetime import datetime
import hashlib
import json
import uuid

print("=" * 80)
print("PRODUCTION ETL - v3.0")
print("=" * 80)
print(f"Spark:    {spark.version}")
print(f"Database: {spark.sql('SELECT current_database()').collect()[0][0]}")
print("=" * 80)

# COMMAND ----------
# MAGIC %md
# MAGIC ## CONFIGURATION

# COMMAND ----------

class Config:
    """Production configuration"""
    DATABASE         = "dbo"
    SOURCE_TABLE     = "person"
    PIPELINE_NAME    = "person_etl_v3"
    ENVIRONMENT      = "PROD"

    # Performance tuning for 45M rows
    SHUFFLE_PARTITIONS = 400
    REPARTITION_COUNT  = 400

    # Schema management
    # FORCE_RECREATE must be set explicitly by a human — never toggled automatically.
    # When False (default): schema mismatches are handled via ALTER TABLE.
    # When True:            DROP + recreate is permitted for full reload scenarios.
    FORCE_RECREATE   = False

    # Compliance
    DATA_CLASSIFICATION   = "CONFIDENTIAL-PERSONAL"
    NHS_VERSION           = "v3.0"
    NHS_UNKNOWN_GENDER    = 8551
    NHS_UNKNOWN_ETHNICITY = 7
    NHS_UNKNOWN_RACE      = 0

    @staticmethod
    def table(name):
        return f"{Config.DATABASE}.{name}"


spark.conf.set("spark.sql.shuffle.partitions", str(Config.SHUFFLE_PARTITIONS))
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

print(f"Config: Partitions={Config.SHUFFLE_PARTITIONS} | FORCE_RECREATE={Config.FORCE_RECREATE}")
print(f"Pipeline: {Config.PIPELINE_NAME} | Env: {Config.ENVIRONMENT}")

# COMMAND ----------
# MAGIC %md
# MAGIC ## UTILITIES

# COMMAND ----------

# Pseudonymization
def pseudonymize(value: str) -> str:
    if not value:
        return None
    return hashlib.sha256(f"{value}FABRIC_2026".encode()).hexdigest()

pseudonymize_udf = F.udf(pseudonymize, StringType())


# ─────────────────────────────────────────────────────────────────────────────
# Schema Inspector — v3.1
#
# PRINCIPLE: A schema change is a governance event, not an automatic fix.
#
# Behaviour:
#   New columns     → ALTER TABLE ADD COLUMNS  (non-breaking, preserves all history)
#   Type conflicts  → Safe cast to existing type + RCA warning
#                     If cast impossible → FAILED (pipeline stops, human action required)
#   FORCE_RECREATE  → DROP only when Config.FORCE_RECREATE = True (human decision)
#   No change       → MERGE (normal incremental path)
#
# All schema events are written to audit_trail (event_type SCHEMA_*).
# Schema errors are written to rca_errors (category SCHEMA).
# Query schema history: SELECT * FROM dbo.audit_trail WHERE event_type LIKE 'SCHEMA%'
#
# What it will NEVER do automatically:
#   - DROP a table because of schema drift
#   - Silently swallow type conflicts
#   - Destroy Delta transaction log history
# ─────────────────────────────────────────────────────────────────────────────
class SchemaInspector:

    @staticmethod
    def _type_str(dtype):
        return dtype.simpleString()

    @staticmethod
    def validate_and_prepare(source_df, table_name, audit, rca, session_id):
        """
        Returns (success: bool, prepared_df: DataFrame, action: str)
        action values: CREATE | MERGE | EVOLVED | RECREATE | FAILED

        Schema events → audit_trail  (event_type SCHEMA_*)
        Schema errors → rca_errors   (category SCHEMA)
        """
        try:
            # ── Table does not exist → CREATE ─────────────────────────────
            if not spark.catalog.tableExists(table_name):
                audit.log("SCHEMA_CHECK",
                          f"Table {table_name} does not exist — will create",
                          status="INFO")
                return True, source_df, "CREATE"

            existing_schema = {f.name: f.dataType for f in spark.table(table_name).schema}
            source_schema   = {f.name: f.dataType for f in source_df.schema}

            new_columns    = {
                c: t for c, t in source_schema.items()
                if c not in existing_schema
            }
            type_conflicts = {
                c: (existing_schema[c], source_schema[c])
                for c in source_schema
                if c in existing_schema
                and SchemaInspector._type_str(existing_schema[c])
                != SchemaInspector._type_str(source_schema[c])
            }

            evolved = False

            # ── Handle type conflicts ─────────────────────────────────────
            if type_conflicts:
                conflict_detail = "; ".join(
                    f"{col}: table={SchemaInspector._type_str(old)} "
                    f"incoming={SchemaInspector._type_str(new)}"
                    for col, (old, new) in type_conflicts.items()
                )
                audit.log("SCHEMA_TYPE_CONFLICT",
                          f"Type conflicts in {table_name}: {conflict_detail}",
                          status="WARNING")
                rca.capture_error(
                    "SCHEMA", "TYPE_CONFLICT", "WARNING", "SCHEMA_VALIDATION",
                    column=", ".join(type_conflicts.keys()),
                    error_value=conflict_detail,
                    expected="Matching data types",
                    rule="SCHEMA_TYPE_COMPATIBILITY",
                    resolution=(
                        "To allow full reload set Config.FORCE_RECREATE=True. "
                        "For column type changes raise a schema migration change request."
                    )
                )

                if Config.FORCE_RECREATE:
                    audit.log("SCHEMA_RECREATE",
                              f"FORCE_RECREATE=True — dropping {table_name}. "
                              f"WARNING: downstream views/semantic models may break.",
                              status="WARNING")
                    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
                    return True, source_df, "RECREATE"

                # Attempt safe cast — cast incoming to match existing table types
                safe_df = source_df
                for col_name, (existing_type, _) in type_conflicts.items():
                    try:
                        safe_df = safe_df.withColumn(
                            col_name, F.col(col_name).cast(existing_type)
                        )
                        audit.log("SCHEMA_CAST",
                                  f"Cast {col_name} to {SchemaInspector._type_str(existing_type)} "
                                  f"to match existing table schema",
                                  status="WARNING")
                        evolved = True
                    except Exception as cast_err:
                        audit.log("SCHEMA_CAST_FAILED",
                                  f"Cannot cast {col_name}: {cast_err} — pipeline stopped. "
                                  f"Raise a schema migration change request.",
                                  status="FAILURE")
                        return False, source_df, "FAILED"
                source_df = safe_df

            # ── Handle new columns — ALTER TABLE (non-breaking) ───────────
            if new_columns:
                for col_name, col_type in new_columns.items():
                    type_str = SchemaInspector._type_str(col_type)
                    try:
                        spark.sql(
                            f"ALTER TABLE {table_name} "
                            f"ADD COLUMNS (`{col_name}` {type_str})"
                        )
                        audit.log("SCHEMA_EVOLVED",
                                  f"Added column `{col_name}` ({type_str}) to {table_name} "
                                  f"— existing rows will have NULL for this column",
                                  status="INFO")
                        evolved = True
                    except Exception as alter_err:
                        audit.log("SCHEMA_ALTER_FAILED",
                                  f"ALTER TABLE failed for `{col_name}`: {alter_err}",
                                  status="FAILURE")
                        rca.capture_error(
                            "SCHEMA", "ALTER_FAILED", "CRITICAL", "SCHEMA_VALIDATION",
                            column=col_name,
                            error_value=str(alter_err),
                            resolution="Check table permissions and Delta Lake version"
                        )
                        return False, source_df, "FAILED"

            # ── Return action ─────────────────────────────────────────────
            if evolved:
                audit.log("SCHEMA_CHECK",
                          f"Schema evolved for {table_name} — proceeding with MERGE",
                          status="SUCCESS")
                return True, source_df, "EVOLVED"

            audit.log("SCHEMA_CHECK",
                      f"Schema compatible — no changes for {table_name}",
                      status="SUCCESS")
            return True, source_df, "MERGE"

        except Exception as e:
            audit.log("SCHEMA_ERROR", f"Schema validation error: {e}", status="FAILURE")
            return False, source_df, "FAILED"


# ─────────────────────────────────────────────────────────────────────────────
# RCA Engine
# ─────────────────────────────────────────────────────────────────────────────
class RCAEngine:
    def __init__(self, session_id):
        self.session_id = session_id
        self.errors     = []

    def capture_error(self, category, error_type, severity, stage, **kwargs):
        self.errors.append({
            "rca_id":      str(uuid.uuid4()),
            "timestamp":   datetime.utcnow(),
            "category":    category,
            "error_type":  error_type,
            "severity":    severity,
            "row_id":      kwargs.get("row_id"),
            "column":      kwargs.get("column"),
            "error_value": str(kwargs.get("error_value")) if kwargs.get("error_value") else None,
            "expected":    kwargs.get("expected"),
            "rule":        kwargs.get("rule"),
            "stage":       stage,
            "session_id":  self.session_id,
            "resolution":  kwargs.get("resolution", "Review error")
        })

    def save(self):
        if not self.errors:
            return
        schema = StructType([
            StructField("rca_id",      StringType(),    False),
            StructField("timestamp",   TimestampType(), False),
            StructField("category",    StringType(),    False),
            StructField("error_type",  StringType(),    False),
            StructField("severity",    StringType(),    False),
            StructField("row_id",      StringType(),    True),
            StructField("column",      StringType(),    True),
            StructField("error_value", StringType(),    True),
            StructField("expected",    StringType(),    True),
            StructField("rule",        StringType(),    True),
            StructField("stage",       StringType(),    False),
            StructField("session_id",  StringType(),    False),
            StructField("resolution",  StringType(),    True)
        ])
        data = [(
            e["rca_id"], e["timestamp"], e["category"], e["error_type"],
            e["severity"], e["row_id"], e["column"], e["error_value"],
            e["expected"], e["rule"], e["stage"], e["session_id"], e["resolution"]
        ) for e in self.errors]
        df        = spark.createDataFrame(data, schema)
        rca_table = Config.table("rca_errors")
        try:
            df.write.mode("append").format("delta").saveAsTable(rca_table)
            return
        except Exception as e1:
            print(f"⚠️ RCA append failed: {e1}")
        try:
            df.write.mode("append").format("delta") \
              .option("mergeSchema", "true").saveAsTable(rca_table)
            print("⚠️ RCA saved via mergeSchema — check for schema drift")
            return
        except Exception as e2:
            print(f"⚠️ RCA mergeSchema failed: {e2}")
        print("❌ WARNING: RCA falling back to overwrite — historical records may be lost")
        try:
            df.write.mode("overwrite").format("delta") \
              .option("overwriteSchema", "true").saveAsTable(rca_table)
        except Exception as e3:
            print(f"❌ CRITICAL: RCA save failed entirely: {e3}")


# ─────────────────────────────────────────────────────────────────────────────
# Audit Logger
# ─────────────────────────────────────────────────────────────────────────────
class AuditLogger:
    def __init__(self, session_id):
        self.session_id = session_id
        self.start_time = datetime.utcnow()
        self.events     = []

    def log(self, event_type, description, stage=None, rows=0, status="SUCCESS", **kwargs):
        duration = float(kwargs.get("duration", 0.0))
        self.events.append({
            "audit_id":         str(uuid.uuid4()),
            "session_id":       self.session_id,
            "timestamp":        datetime.utcnow(),
            "event_type":       event_type,
            "description":      description,
            "stage":            stage,
            "rows":             int(rows),
            "status":           status,
            "duration_seconds": duration,
            "metadata":         json.dumps(kwargs.get("metadata", {}))
        })
        icon = "✅" if status == "SUCCESS" else "⚠️" if status == "WARNING" \
               else "❌" if status == "FAILURE" else "ℹ️"
        print(f"{icon} {event_type}: {description}")

    def save(self):
        """
        Persists audit events to dbo.audit_trail.

        Type compatibility strategy (v3.2):
        ─────────────────────────────────
        audit_trail was created in v1 with rows as IntegerType (INT).
        The incoming DataFrame now declares rows as LongType (BIGINT).

        ALTER TABLE CHANGE COLUMN for type widening is unsupported in the
        Delta Lake protocol version running on this cluster. mergeSchema=true
        handles additive schema changes only — it does not resolve type
        conflicts. Both approaches fail, as observed in production.

        Correct resolution: detect the existing column type at runtime and
        align the outgoing DataFrame schema to match it before writing.
        This eliminates the conflict without DDL surgery or overwrite fallback.

        When the table does not yet exist (first run), rows is written as
        IntegerType to establish a consistent baseline. INT is safe at the
        current 2.7M row scale. A planned FORCE_RECREATE during a maintenance
        window is the appropriate upgrade path when scale approaches 2.1B rows.
        """
        if not self.events:
            return

        audit_table = Config.table("audit_trail")

        # Determine the rows column type in the existing table, if present.
        # Default to IntegerType to match the baseline established in v1.
        if spark.catalog.tableExists(audit_table):
            existing_rows_fields = [
                f.dataType for f in spark.table(audit_table).schema
                if f.name == "rows"
            ]
            rows_type = existing_rows_fields[0] if existing_rows_fields else IntegerType()
        else:
            rows_type = IntegerType()

        schema = StructType([
            StructField("audit_id",         StringType(),    False),
            StructField("session_id",       StringType(),    False),
            StructField("timestamp",        TimestampType(), False),
            StructField("event_type",       StringType(),    False),
            StructField("description",      StringType(),    False),
            StructField("stage",            StringType(),    True),
            StructField("rows",             rows_type,       True),  # matched to existing table
            StructField("status",           StringType(),    False),
            StructField("duration_seconds", DoubleType(),    True),
            StructField("metadata",         StringType(),    True)
        ])

        # Cast rows values to match the target type (int or long) before DataFrame creation.
        # Python int is compatible with both IntegerType and LongType in PySpark.
        data = [(
            e["audit_id"], e["session_id"], e["timestamp"], e["event_type"],
            e["description"], e["stage"], int(e["rows"]), e["status"],
            e["duration_seconds"], e["metadata"]
        ) for e in self.events]

        df = spark.createDataFrame(data, schema)

        # Standard append — no mergeSchema required because the schema is now
        # guaranteed to match the existing table exactly.
        try:
            df.write.mode("append").format("delta").saveAsTable(audit_table)
        except Exception as e1:
            # Second attempt: mergeSchema handles the case where other columns
            # have drifted (e.g. new columns added by a future pipeline version).
            print(f"⚠️ AUDIT_TRAIL: Standard append failed ({e1}) — retrying with mergeSchema")
            try:
                df.write.mode("append").format("delta") \
                  .option("mergeSchema", "true").saveAsTable(audit_table)
            except Exception as e2:
                # Last resort — explicit, never silent.
                # This path should not be reachable under normal operations.
                print(f"❌ AUDIT_TRAIL: mergeSchema append failed ({e2})")
                print("❌ AUDIT_TRAIL: WARNING — falling back to overwrite. "
                      "Historical audit records will be lost. "
                      "Investigate root cause before next pipeline execution.")
                df.write.mode("overwrite").format("delta") \
                  .option("overwriteSchema", "true").saveAsTable(audit_table)

    def get_summary(self):
        duration = (datetime.utcnow() - self.start_time).total_seconds()
        return {
            "session_id": self.session_id,
            "duration":   duration,
            "events":     len(self.events),
            "success":    sum(1 for e in self.events if e["status"] == "SUCCESS"),
            "failure":    sum(1 for e in self.events if e["status"] == "FAILURE")
        }


# ─────────────────────────────────────────────────────────────────────────────
# Data Quality
# ─────────────────────────────────────────────────────────────────────────────
def apply_dq_checks(df, rules, audit):
    df_dq = df.withColumn("dq_status",   F.lit("VALID")) \
              .withColumn("dq_failures", F.array().cast(ArrayType(StringType())))

    for rule in rules:
        df_dq = df_dq \
            .withColumn("dq_status",
                F.when(~rule["condition"], F.lit("ERROR"))
                 .otherwise(F.col("dq_status"))) \
            .withColumn("dq_failures",
                F.when(~rule["condition"],
                       F.array_union(F.col("dq_failures"), F.array(F.lit(rule["name"]))))
                .otherwise(F.col("dq_failures")))

    valid_df      = df_dq.filter(F.col("dq_status") == "VALID")
    quarantine_df = df_dq.filter(F.col("dq_status") != "VALID")
    total         = df.count()
    valid         = valid_df.count()
    pass_rate     = round((valid / total) * 100, 2) if total > 0 else 0

    audit.log("DQ_VALIDATION", f"Pass rate: {pass_rate}%", "SILVER", total)
    return valid_df, quarantine_df, {"total": total, "valid": valid, "pass_rate": pass_rate}


# ─────────────────────────────────────────────────────────────────────────────
# NHS Rules
# ─────────────────────────────────────────────────────────────────────────────
def apply_nhs_rules(df):
    df = df.withColumn("gender_concept_id_clean",
            F.when(F.col("gender_concept_id").isNull(), F.lit(Config.NHS_UNKNOWN_GENDER))
             .when(~F.col("gender_concept_id").isin([8507, 8532]), F.lit(Config.NHS_UNKNOWN_GENDER))
             .otherwise(F.col("gender_concept_id")))

    df = df.withColumn("race_concept_id_clean",
            F.when(F.col("race_concept_id").isNull(), F.lit(Config.NHS_UNKNOWN_RACE))
             .otherwise(F.col("race_concept_id")))

    df = df.withColumn("ethnicity_concept_id_clean",
            F.when(F.col("ethnicity_concept_id").isNull(), F.lit(Config.NHS_UNKNOWN_ETHNICITY))
             .when(F.col("ethnicity_concept_id") == 0, F.lit(Config.NHS_UNKNOWN_ETHNICITY))
             .otherwise(F.col("ethnicity_concept_id")))

    df = df.withColumn("birth_date",
            F.when(
                F.col("year_of_birth").isNotNull() &
                F.col("month_of_birth").isNotNull() &
                F.col("day_of_birth").isNotNull(),
                F.make_date(F.col("year_of_birth"), F.col("month_of_birth"), F.col("day_of_birth"))
            ).otherwise(None))

    df = df.withColumn("age_years",
            F.floor(F.months_between(F.current_date(), F.col("birth_date")) / 12))

    df = df.withColumn("nhs_age_band",
            F.when(F.col("age_years") < 1,               "0-<1")
             .when(F.col("age_years").between(1,  4),     "1-4")
             .when(F.col("age_years").between(5,  9),     "5-9")
             .when(F.col("age_years").between(10, 14),    "10-14")
             .when(F.col("age_years").between(15, 19),    "15-19")
             .when(F.col("age_years").between(20, 24),    "20-24")
             .when(F.col("age_years").between(25, 29),    "25-29")
             .when(F.col("age_years").between(30, 34),    "30-34")
             .when(F.col("age_years").between(35, 39),    "35-39")
             .when(F.col("age_years").between(40, 44),    "40-44")
             .when(F.col("age_years").between(45, 49),    "45-49")
             .when(F.col("age_years").between(50, 54),    "50-54")
             .when(F.col("age_years").between(55, 59),    "55-59")
             .when(F.col("age_years").between(60, 64),    "60-64")
             .when(F.col("age_years").between(65, 69),    "65-69")
             .when(F.col("age_years").between(70, 74),    "70-74")
             .when(F.col("age_years").between(75, 79),    "75-79")
             .when(F.col("age_years").between(80, 84),    "80-84")
             .when(F.col("age_years") >= 85,              "85+")
             .otherwise("Unknown"))

    df = df.withColumn("ecds_compliant", F.lit(True)) \
           .withColumn("ecds_version",   F.lit(Config.NHS_VERSION))
    return df


print("✅ Utilities loaded")

# COMMAND ----------
# MAGIC %md
# MAGIC ## MAIN ETL PIPELINE — v3.0

# COMMAND ----------

def run_production_etl():

    session_id = str(uuid.uuid4())

    print("\n" + "=" * 80)
    print("PRODUCTION ETL v3.2 — EXECUTION")
    print("=" * 80)
    print(f"Session:       {session_id}")
    print(f"Pipeline:      {Config.PIPELINE_NAME}")
    print(f"Environment:   {Config.ENVIRONMENT}")
    print(f"ForceRecreate: {Config.FORCE_RECREATE}")
    print("=" * 80)

    audit     = AuditLogger(session_id)
    rca       = RCAEngine(session_id)
    inspector = SchemaInspector()

    audit.log("PIPELINE_START", f"ETL v3.2 started | {Config.PIPELINE_NAME}", "INIT")

    table_actions = {}

    try:
        # =================================================================
        # BRONZE — Raw Ingestion
        # =================================================================
        print("\n[BRONZE] Raw ingestion...")
        start_time = datetime.utcnow()

        source_df = spark.table(Config.table(Config.SOURCE_TABLE))

        bronze_df = source_df \
            .withColumn("ingestion_timestamp",  F.current_timestamp()) \
            .withColumn("pipeline_run_id",      F.lit(session_id)) \
            .filter(F.col("person_id").isNotNull()) \
            .withColumn("lineage_source",       F.lit(f"dbo.{Config.SOURCE_TABLE}")) \
            .withColumn("lineage_pipeline",     F.lit(Config.PIPELINE_NAME)) \
            .withColumn("lineage_environment",  F.lit(Config.ENVIRONMENT)) \
            .withColumn("lineage_bronze_ts",    F.current_timestamp())

        bronze_df    = bronze_df.repartition(Config.REPARTITION_COUNT)
        bronze_count = bronze_df.count()
        audit.log("BRONZE_LOADED",
                  f"Loaded {bronze_count:,} records from {Config.SOURCE_TABLE}",
                  "BRONZE", bronze_count)

        bronze_table = Config.table("bronze_person")
        success, prepared_df, action = inspector.validate_and_prepare(
            bronze_df, bronze_table, audit, rca, session_id
        )
        table_actions["bronze_person"] = action

        if not success:
            raise ValueError("Bronze schema validation failed — check rca_errors table")

        if action in ["CREATE", "RECREATE"]:
            prepared_df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(bronze_table)
            audit.log("BRONZE_WRITE", f"{action}: {bronze_table}", "BRONZE", bronze_count)
        else:
            # FIX 1: MERGE key is person_id only — no pipeline_run_id
            target = DeltaTable.forName(spark, bronze_table)
            target.alias("target").merge(
                prepared_df.alias("source"),
                "target.person_id = source.person_id"
            ).whenMatchedUpdateAll() \
             .whenNotMatchedInsertAll() \
             .execute()
            audit.log("BRONZE_MERGE", f"Merged into {bronze_table}", "BRONZE", bronze_count)

        spark.sql(f"OPTIMIZE {bronze_table}")

        end_time   = datetime.utcnow()
        duration   = (end_time - start_time).total_seconds()
        throughput = bronze_count / duration if duration > 0 else 0
        audit.log("BRONZE_COMPLETE",
                  f"Bronze complete: {bronze_count:,} records in {duration:.2f}s ({throughput:.0f} rows/s)",
                  "BRONZE", bronze_count, duration=duration)

        # =================================================================
        # SILVER — Validation & Enrichment
        # =================================================================
        print("\n[SILVER] Validation & enrichment...")
        start_time = datetime.utcnow()

        dq_rules = [
            {"name": "PERSON_ID_NOT_NULL", "condition": F.col("person_id").isNotNull()},
            {"name": "GENDER_VALID", "condition":
                F.col("gender_concept_id").isin([8507, 8532, 8551]) |
                F.col("gender_concept_id").isNull()},
            {"name": "BIRTH_YEAR_RANGE", "condition":
                F.col("year_of_birth").between(1900, 2026) |
                F.col("year_of_birth").isNull()}
        ]

        silver_valid_df, quarantine_df, dq_metrics = apply_dq_checks(
            bronze_df, dq_rules, audit
        )

        print(f"   DQ Pass Rate: {dq_metrics['pass_rate']}%")
        print(f"   Valid: {dq_metrics['valid']:,} | Quarantine: {dq_metrics['total'] - dq_metrics['valid']:,}")

        silver_df = apply_nhs_rules(silver_valid_df)

        if "person_source_value" in silver_df.columns:
            silver_df = silver_df.withColumn(
                "person_source_value_pseudo",
                pseudonymize_udf(F.col("person_source_value"))
            )
            audit.log("PSEUDONYMIZATION", "Applied GDPR pseudonymization", "SILVER")

        silver_df = silver_df \
            .withColumn("silver_timestamp",  F.current_timestamp()) \
            .withColumn("lineage_silver_ts", F.current_timestamp())

        silver_df    = silver_df.repartition(Config.REPARTITION_COUNT)
        silver_table = Config.table("silver_person")
        success, prepared_df, action = inspector.validate_and_prepare(
            silver_df, silver_table, audit, rca, session_id
        )
        table_actions["silver_person"] = action

        if not success:
            raise ValueError("Silver schema validation failed — check rca_errors table")

        if action in ["CREATE", "RECREATE"]:
            prepared_df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(silver_table)
        else:
            target = DeltaTable.forName(spark, silver_table)
            target.alias("target").merge(
                prepared_df.alias("source"),
                "target.person_id = source.person_id"
            ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

        spark.sql(f"OPTIMIZE {silver_table}")

        end_time = datetime.utcnow()
        audit.log("SILVER_COMPLETE",
                  f"Silver complete: {dq_metrics['valid']:,} records",
                  "SILVER", dq_metrics['valid'],
                  duration=(end_time - start_time).total_seconds())

        if dq_metrics['total'] - dq_metrics['valid'] > 0:
            quarantine_table = Config.table(
                f"quarantine_person_{datetime.now().strftime('%Y%m%d')}"
            )
            quarantine_df.write.mode("append").format("delta").saveAsTable(quarantine_table)
            audit.log("QUARANTINE_SAVED",
                      f"Quarantined {dq_metrics['total'] - dq_metrics['valid']:,} records",
                      "SILVER")

        # =================================================================
        # GOLD — Business Layer
        # =================================================================
        print("\n[GOLD] Business layer...")
        start_time = datetime.utcnow()

        if "person_source_value_pseudo" in silver_df.columns:
            person_key_col = F.col("person_source_value_pseudo").alias("person_key")
            person_key_src = "person_source_value_pseudo"
        else:
            person_key_col = F.col("person_id").cast(StringType()).alias("person_key")
            person_key_src = "person_id (fallback — person_source_value not present)"

        audit.log("GOLD_KEY", f"person_key sourced from: {person_key_src}", "GOLD")

        gold_df = silver_df.select(
            F.col("person_id"),
            person_key_col,
            F.col("gender_concept_id_clean").alias("gender_concept_id"),
            F.col("age_years"),
            F.col("nhs_age_band"),
            F.col("ecds_compliant"),
            F.col("lineage_source"),
            F.col("lineage_pipeline"),
            F.col("lineage_environment"),
            F.col("lineage_bronze_ts"),
            F.col("lineage_silver_ts"),
            F.current_timestamp().alias("lineage_gold_ts"),
            F.lit(session_id).alias("lineage_session_id"),
            F.current_timestamp().alias("gold_created")
        )

        gold_df    = gold_df.repartition(Config.REPARTITION_COUNT)
        gold_table = Config.table("gold_person")
        success, prepared_df, action = inspector.validate_and_prepare(
            gold_df, gold_table, audit, rca, session_id
        )
        table_actions["gold_person"] = action

        if not success:
            raise ValueError("Gold schema validation failed — check rca_errors table")

        if action in ["CREATE", "RECREATE"]:
            prepared_df.write.format("delta").mode("overwrite") \
                .option("overwriteSchema", "true").saveAsTable(gold_table)
        else:
            target = DeltaTable.forName(spark, gold_table)
            target.alias("target").merge(
                prepared_df.alias("source"),
                "target.person_id = source.person_id"
            ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

        spark.sql(f"OPTIMIZE {gold_table}")
        gold_count = spark.table(gold_table).count()

        end_time = datetime.utcnow()
        audit.log("GOLD_COMPLETE", f"Gold complete: {gold_count:,} records",
                  "GOLD", gold_count,
                  duration=(end_time - start_time).total_seconds())

        # =================================================================
        # DIMENSION — SCD Type 2 (proper two-step expire + insert)
        # =================================================================
        print("\n[DIM] Dimension (SCD Type 2)...")
        start_time = datetime.utcnow()

        dim_new_df = gold_df.select(
            F.col("person_id"),
            F.col("person_key"),
            F.col("gender_concept_id"),
            F.col("age_years"),
            F.col("nhs_age_band"),
            F.col("ecds_compliant"),
            F.col("lineage_session_id")
        ).withColumn("effective_from", F.current_date()) \
         .withColumn("effective_to",   F.lit("9999-12-31").cast("date")) \
         .withColumn("is_current",     F.lit(True))

        dim_table = Config.table("dim_person")
        success, prepared_df, action = inspector.validate_and_prepare(
            dim_new_df, dim_table, audit, rca, session_id
        )
        table_actions["dim_person"] = action

        if not success:
            raise ValueError("Dimension schema validation failed — check rca_errors table")

        if action in ["CREATE", "RECREATE"]:
            prepared_df.write.format("delta").mode("overwrite") \
                .option("overwriteSchema", "true").saveAsTable(dim_table)
            audit.log("DIM_CREATE", f"{action}: {dim_table}", "DIM")
        else:
            # FIX 2: Proper SCD2 — two-step expire then insert
            target = DeltaTable.forName(spark, dim_table)

            # Step 1: Expire changed current records
            target.alias("target").merge(
                prepared_df.alias("source"),
                "target.person_id = source.person_id AND target.is_current = true"
            ).whenMatchedUpdate(
                condition="""
                    target.gender_concept_id != source.gender_concept_id OR
                    target.nhs_age_band       != source.nhs_age_band      OR
                    target.ecds_compliant     != source.ecds_compliant
                """,
                set={
                    "is_current":   F.lit(False),
                    "effective_to": F.current_date()
                }
            ).execute()
            audit.log("DIM_SCD2_EXPIRE",
                      "Step 1: expired changed current records", "DIM")

            # Step 2: Insert new current versions for changed + new persons
            target = DeltaTable.forName(spark, dim_table)
            target.alias("target").merge(
                prepared_df.alias("source"),
                "target.person_id = source.person_id AND target.is_current = true"
            ).whenNotMatchedInsertAll().execute()
            audit.log("DIM_SCD2_INSERT",
                      "Step 2: inserted new current versions", "DIM")

        spark.sql(f"OPTIMIZE {dim_table}")
        dim_count         = spark.table(dim_table).count()
        dim_current_count = spark.table(dim_table).filter(F.col("is_current") == True).count()
        dim_expired_count = dim_count - dim_current_count

        end_time = datetime.utcnow()
        audit.log("DIM_COMPLETE",
                  f"Dimension complete: {dim_count:,} total | "
                  f"{dim_current_count:,} current | {dim_expired_count:,} expired",
                  "DIM", dim_count,
                  duration=(end_time - start_time).total_seconds())

        # =================================================================
        # SUMMARY
        # =================================================================
        total_duration = (datetime.utcnow() - audit.start_time).total_seconds()

        print("\n" + "=" * 80)
        print("✅✅✅ PIPELINE SUCCESS ✅✅✅")
        print("=" * 80)
        print(f"Session:     {session_id}")
        print(f"Duration:    {total_duration:.2f}s")
        print(f"Bronze:      {bronze_count:,}")
        print(f"Silver:      {dq_metrics['valid']:,} (DQ: {dq_metrics['pass_rate']}%)")
        print(f"Gold:        {gold_count:,}")
        print(f"Dimension:   {dim_count:,} total | {dim_current_count:,} current | {dim_expired_count:,} expired")
        print(f"Throughput:  {int(bronze_count / total_duration):,} rows/s")
        print(f"NHS ECDS:    {Config.NHS_VERSION} ✅ | GDPR: Pseudonymized ✅")
        print(f"Actions:     {table_actions}")
        print("=" * 80)

        audit.log("PIPELINE_COMPLETE", "Pipeline completed successfully", "COMPLETE",
                  metadata={
                      "bronze": bronze_count, "silver": dq_metrics['valid'],
                      "gold": gold_count, "dimension": dim_count,
                      "dim_current": dim_current_count, "dim_expired": dim_expired_count,
                      "duration": total_duration, "table_actions": table_actions
                  })

    except Exception as e:
        audit.log("PIPELINE_FAILURE", f"Pipeline failed: {str(e)}", status="FAILURE")
        rca.capture_error("SYSTEM", type(e).__name__, "CRITICAL", "PIPELINE",
                          error_value=str(e), resolution="Review logs and rca_errors table")
        print(f"\n❌ Pipeline failed: {str(e)}")

    finally:
        audit.save()
        rca.save()
        summary = audit.get_summary()
        print(f"\n📊 Session Summary:")
        print(f"   Duration: {summary['duration']:.2f}s")
        print(f"   Events:   {summary['events']}")
        print(f"   Success:  {summary['success']} | Failures: {summary['failure']}")


# COMMAND ----------

# RUN PIPELINE
run_production_etl()

# COMMAND ----------
# MAGIC %md
# MAGIC ## VERIFICATION

# COMMAND ----------

print("\n📊 TABLE RECORD COUNTS (this run):")
print(f"   {'Table':<40} {'Records':>15}   Notes")
print("   " + "-" * 72)

tables = spark.sql(f"SHOW TABLES IN {Config.DATABASE}").filter(
    F.col("tableName").like("%person%") |
    F.col("tableName").like("%audit%")  |
    F.col("tableName").like("%rca%")
).collect()

for t in tables:
    full_name = f"{Config.DATABASE}.{t.tableName}"
    count     = spark.table(full_name).count()
    note      = ""
    if t.tableName == "dim_person":
        current = spark.table(full_name).filter(F.col("is_current") == True).count()
        expired = count - current
        note    = f"({current:,} current | {expired:,} expired)"
    elif t.tableName == "bronze_person":
        note    = "(should match source count — watch for drift)"
    print(f"   {t.tableName:<40} {count:>15,}   {note}")

print("\n📋 Schema change history — from audit_trail:")
display(
    spark.table(Config.table("audit_trail"))
         .filter(F.col("event_type").like("SCHEMA%"))
         .orderBy(F.col("timestamp").desc())
         .select("timestamp", "event_type", "status", "description", "session_id")
)

# COMMAND ----------
# MAGIC %md
# MAGIC ---
# MAGIC ## ETL v3.2 — Change Summary
# MAGIC
# MAGIC **Single change in this increment — audit_trail rows type conflict:**
# MAGIC
# MAGIC | | v3.1 (failed) | v3.2 (correct) |
# MAGIC |---|---|---|
# MAGIC | Approach | ALTER TABLE CHANGE COLUMN INT → BIGINT | Detect existing type at runtime, align outgoing schema to match |
# MAGIC | Why v3.1 failed | ALTER TABLE CHANGE COLUMN unsupported in this Delta protocol version | N/A |
# MAGIC | Why mergeSchema failed | mergeSchema handles additive changes only — not type conflicts | N/A |
# MAGIC | Fallback | Overwrote audit_trail — destroyed history | Not reachable under normal operations |
# MAGIC | rows type in table | INT (established in v1, cannot be widened via DDL here) | INT — matched at write time |
# MAGIC | rows type in DataFrame | LongType (v2/v3 declaration) | Dynamically matched to existing table |
# MAGIC | Overflow risk | At 2.1B rows | Same — planned FORCE_RECREATE is the upgrade path at scale |
# MAGIC
# MAGIC **Bronze table strategy — on record:**
# MAGIC The MERGE on `person_id` is the correct pattern for a full-extract source.
# MAGIC Source deletions (records absent from the current extract) are not yet handled.
# MAGIC Orphaned Bronze rows from deleted source records accumulate silently.
# MAGIC This is tracked as a pending item for v4.0 (soft-delete via `is_deleted` flag).
# MAGIC
# MAGIC **Pending — next iterations:**
# MAGIC - v3.3: DQ reads from persisted bronze table + expand NHS ECDS rule set + apply_nhs_rules() error handling
# MAGIC - v4.0: Source deletion handling in Bronze, quarantine retention, location context, idempotency guard


StatementMeta(, eb6b5536-2ea7-457e-867b-ffe1dc16b305, 3, Finished, Available, Finished, False)

PRODUCTION ETL - v3.0
Spark:    3.5.5.5.4.20260109.1
Database: chimcobldhq2al3id5gmo9acc5lmachk4li64ro
Config: Partitions=400 | FORCE_RECREATE=False
Pipeline: person_etl_v3 | Env: PROD
✅ Utilities loaded

PRODUCTION ETL v3.2 — EXECUTION
Session:       88105352-6749-46e8-878a-87333e00a5d5
Pipeline:      person_etl_v3
Environment:   PROD
ForceRecreate: False
✅ PIPELINE_START: ETL v3.2 started | person_etl_v3

[BRONZE] Raw ingestion...
✅ BRONZE_LOADED: Loaded 2,712,818 records from person
✅ SCHEMA_CHECK: Schema compatible — no changes for dbo.bronze_person
✅ BRONZE_MERGE: Merged into dbo.bronze_person
✅ BRONZE_COMPLETE: Bronze complete: 2,712,818 records in 88.98s (30488 rows/s)

[SILVER] Validation & enrichment...
✅ DQ_VALIDATION: Pass rate: 100.0%
   DQ Pass Rate: 100.0%
   Valid: 2,712,818 | Quarantine: 0
✅ PSEUDONYMIZATION: Applied GDPR pseudonymization
✅ SCHEMA_CHECK: Schema compatible — no changes for dbo.silver_person
✅ SILVER_COMPLETE: Silver complete: 2,712,818 records

[GOLD] Bu

SynapseWidget(Synapse.DataFrame, 0e51391a-33fd-4ca5-af53-461793ced782)